In [ ]:
# !pip install selenium

In [3]:
# !docker-compose up -d

In [1]:
# from ThreadWorker import ThreadWorker
from ProxyLoader import ProxyLoader
from LetudiantScraping import LetudiantScraping
from EcoleStorage import EcoleStorage
from Ecole import Ecole

import pandas as pd
import threading
import re

proxyLoader = ProxyLoader()
proxy = proxyLoader.get_proxy(51)

regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Bretagne',
    'Centre-Val de Loire',
    'Corse',
    'Grand Est',
    'Hauts-de-France',
    "Normandie"
]

region = "Normandie"
file_path = 'results/ecoles_results_7'
is_for_more_inf = False
def main():

    # first part
    if not is_for_more_inf:
        letudiantScraping = LetudiantScraping(url="https://www.letudiant.fr/etudes/annuaire-enseignement-superieur/etablissement/critere-Master.html", region=region, file_path=file_path, with_selenium_grid=False)
        result = letudiantScraping.start_scraping()
        print(result)
    # second part    
    try:
        # Lire le fichier CSV de la liste des ecoles
        df = pd.read_csv(f'{file_path}.csv')
        df['ecole_address'] = None
        df['ecole_comment_contacter'] = None
        ecoles = []
        for index, row in df.iterrows():
            ecole = Ecole()
            ecole.init_from_dic(row)
            letudiantScraping = LetudiantScraping(url=row['ecole_more_inf_url'], proxy=proxy, with_selenium_grid=False)
            result = letudiantScraping.start_scraping_more_info()
            if result['status']:
                ecole.ecole_phone = result['data'].ecole_phone
                ecole.ecole_email = result['data'].ecole_email
                ecole.ecole_address = result['data'].ecole_address
                ecole.ecole_web_site_url = result['data'].ecole_web_site_url
                ecole.ecole_comment_contacter = result['data'].ecole_comment_contacter
                ecoles.append(ecole)
                
            if index % 10 == 0:
                ecoleStorage = EcoleStorage(f'{file_path}_{nom_valide(region)}')
                ecoleStorage.insert_ecoles(ecoles)
                ecoleStorage.close_file()
                print(f'new storage : {len(ecoles)} => at index: {index}')
                ecoles = []

        if len(ecoles) > 0:
            ecoleStorage = EcoleStorage(f'{file_path}_{nom_valide(region)}')
            ecoleStorage.insert_ecoles(ecoles)
            ecoleStorage.close_file()
            print(f'new storage : {len(ecoles)} => at index: {index}')
            ecoles = []
            
    except Exception as e:
        print(e)
        if len(ecoles) > 0:
            ecoleStorage = EcoleStorage(f'{file_path}_{nom_valide(region)}')
            ecoleStorage.insert_ecoles(ecoles)
            ecoleStorage.close_file()
            print(f'new storage : {len(ecoles)} => at index: {index}')
            ecoles = []

def nom_valide(chaine):
    chaine = chaine.strip()
    # Remplacer les espaces multiples par un seul tiret bas
    chaine = re.sub(r'\s+', '_', chaine)
    # Supprimer les caractères non autorisés (garde les lettres, chiffres, tirets bas et points)
    chaine = re.sub(r'[^\w.-]', '_', chaine)
    return chaine
     
if __name__ == "__main__":
    main()
    # Lire le fichier CSV
    df = pd.read_csv(f'{file_path}_{nom_valide(region)}.csv')
    # Enregistrer dans un fichier Excel
    df.to_excel(f'{file_path}_{nom_valide(region)}.xlsx', index=False, engine='openpyxl')

https://www.letudiant.fr/etudes/annuaire-enseignement-superieur/etablissement/critere-Master/region-normandie.html
******************************************************************************************************************************************************
le nombre des ecoles est :  20
https://www.letudiant.fr/etudes/annuaire-enseignement-superieur/etablissement/critere-Master/region-normandie/page-2.html
******************************************************************************************************************************************************
le nombre des ecoles est :  18
3333333333
None
new storage : 1 => at index: 0
new storage : 10 => at index: 10
new storage : 10 => at index: 20
new storage : 10 => at index: 30
new storage : 7 => at index: 37
